In [27]:
# 앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.
# 이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").
# 항상 세 개의 이모티콘으로 답장하도록 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하여 체인에 예시를 제공하세요.
# 메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.


from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.memory import ConversationSummaryBufferMemory

llm = ChatOpenAI(
    temperature=0.1, 
    model="gpt-3.5-turbo",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="history",
    return_messages=True,
)

examples=[
    {
       "question": "영화 아바타에 대해 알려줘.",
       "answer": """
            아바타 (2009)
            시놉시스: 🌌🌿🤝
       """,
   },
   {
       "question": "영화 인셉션에 대해 알려줘.",
       "answer": """
            인셉션 (2010)
            시놉시스: 🌀💤🕰️
        """,
   },
   {
       "question": "영화 라이온킹에 대해 알려줘.",
       "answer": """
            라이온킹 (1994)
            시놉시스: 🦁👑🌅
        """,
   }
]


example_template = ChatPromptTemplate.from_messages([
    ("human", "{question}"),
    ("ai","{answer}"),
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_template,
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system","너는 영화 매니아야"),
    MessagesPlaceholder(variable_name="history"),
    example_prompt,
    ("human","{movieTitle}"),
])

chain = final_prompt | llm

def invoke_chain(question):
    result = chain.invoke({
        "history": memory.load_memory_variables({})['history'],
        "movieTitle":question
    })
    memory.save_context(
        {"input":question},
        {"output":result.content}
    )
    print(result)


In [28]:
# memory.load_memory_variables({})['history']

invoke_chain("아저씨")

아저씨 (2010)
시놉시스: 🥋🔪💥content='아저씨 (2010)\n시놉시스: 🥋🔪💥'


In [29]:
invoke_chain("오징어 게임")


오징어 게임 (2021)
시놉시스: 🦑💰🎮content='오징어 게임 (2021)\n시놉시스: 🦑💰🎮'


In [31]:
invoke_chain("내가 물어본 영화 제목 알려줘.")

미안해요. 다시 알려드릴게요.

아바타 (2009)
인셉션 (2010)
라이온킹 (1994)The human and AI discuss movies, mentioning "아저씨" and "오징어 게임." The human asks the AI to recommend a movie they have not mentioned. The AI apologizes and recommends "아바타 (2009)," a science fiction fantasy film set on the planet Pandora where a war between humans and the Na'vi unfolds.content='미안해요. 다시 알려드릴게요.\n\n아바타 (2009)\n인셉션 (2010)\n라이온킹 (1994)'
